### Share secrets with ECDH

In [1]:
import hashlib
import random 

Pcurve = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff  # The proven prime
N = 0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551  # Number of points in the field
Acurve = 0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc
Bcurve = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b  # This defines the curve. y^2 = x^3 + Acurve * x + Bcurve
Gx = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
Gy = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5
GPoint = (Gx, Gy)  # This is our generator point. Trillions of different ones possible

def modinv(a, n=Pcurve):  # Extended Euclidean Algorithm/'division' in elliptic curves
    lm, hm = 1, 0
    low, high = a % n, n
    while low > 1:
        ratio = high // low  # Use integer division operator for Python 3.x
        nm, new = hm - lm * ratio, high - low * ratio
        lm, low, hm, high = nm, new, lm, low
    return lm % n

def ECadd(xp, yp, xq, yq):  # Not true addition, invented for EC. It adds Point-P with Point-Q.
    m = ((yq - yp) * modinv(xq - xp, Pcurve)) % Pcurve
    xr = (m * m - xp - xq) % Pcurve
    yr = (m * (xp - xr) - yp) % Pcurve
    return (xr, yr)

def ECdouble(xp, yp):  # EC point doubling, invented for EC. It doubles Point-P.
    LamNumer = 3 * xp * xp + Acurve
    LamDenom = 2 * yp
    Lam = (LamNumer * modinv(LamDenom, Pcurve)) % Pcurve
    xr = (Lam * Lam - 2 * xp) % Pcurve
    yr = (Lam * (xp - xr) - yp) % Pcurve
    return (xr, yr)

def EccMultiply(xs, ys, Scalar):  # Double & add. EC Multiplication, Not true multiplication
    if Scalar == 0 or Scalar >= N:
        raise Exception("Invalid Scalar/Private Key")
    ScalarBin = str(bin(Scalar))[2:]
    Qx, Qy = xs, ys
    for i in range(1, len(ScalarBin)):  # This is invented EC multiplication.
        Qx, Qy = ECdouble(Qx, Qy)
        if ScalarBin[i] == "1":
            Qx, Qy = ECadd(Qx, Qy, xs, ys)
    return (Qx, Qy)


In [2]:
# Generate private and public keys for Party A
privKeyA = random.randint(1, N-1)
xPublicKeyA, yPublicKeyA = EccMultiply(Gx, Gy, privKeyA % N)

# Generate private and public keys for Party B
privKeyB = random.randint(1, N-1)
xPublicKeyB, yPublicKeyB = EccMultiply(Gx, Gy, privKeyB % N)

# Compute the shared secret for Party A
sharedSecretA = EccMultiply(xPublicKeyB, yPublicKeyB, privKeyA)
sharedSecretAx, sharedSecretAy = sharedSecretA

# Compute the shared secret for Party B
sharedSecretB = EccMultiply(xPublicKeyA, yPublicKeyA, privKeyB)
sharedSecretBx, sharedSecretBy = sharedSecretB

shared_secret = hashlib.sha256(hex(sharedSecretAx)[2:].encode()).digest()

In [15]:
hex(sharedSecretBx)

'0x788086096da9e2f75ea0b93dca147c0dc4ade62831f77b38cad6d993a5759b47'

In [3]:
sharedSecretA

(54504623342482516819070788745766537454710275242106424694645068021476514700103,
 82100628913547009872984417877281507710432614046070068336902694963182840008436)

In [4]:
sharedSecretB

(54504623342482516819070788745766537454710275242106424694645068021476514700103,
 82100628913547009872984417877281507710432614046070068336902694963182840008436)

In [5]:
# Ensure that the shared secrets computed by both parties match
assert sharedSecretAx == sharedSecretBx and sharedSecretAy == sharedSecretBy #this halts the code if the condition is not met

In [6]:
print("Party A:")
print("Private key (hex):", hex(privKeyA))
print("Public key (x, y):", hex(xPublicKeyA), hex(yPublicKeyA))
print("\nParty B:")
print("Private key (hex):", hex(privKeyB))
print("Public key (x, y):", hex(xPublicKeyB), hex(yPublicKeyB))
print("\nShared Secret A (x, y):", hex(sharedSecretAx), hex(sharedSecretAy))
print("\nShared Secret B (x, y):", hex(sharedSecretBx), hex(sharedSecretBy))

print("\nShared secret: ", shared_secret.hex())

Party A:
Private key (hex): 0x675cc409ca37a996d1a152b4886e676b2ee29dcb5d5635fe89353b4a637bc11a
Public key (x, y): 0xa64faf5d1f77cf95e6fd99f45a1a563d978eda88635ca24c1fc9ec8118d2b775 0x243658188c47867bf31d5ff1dbdec17e822345a2307482d908467c6242b845e3

Party B:
Private key (hex): 0xde98d75d7664af7131049907e14199e8a55af6a0d274710ea33464c96e408e6a
Public key (x, y): 0x1de3a8b54ee4c5cb11152aa78e06e8880f096f9e2f4e39353b4aae9c3ee85662 0x92dd4afdbac0315bdec856ef3cf5735d4ae5312f2f7d86f69e44bcf8f5e5f388

Shared Secret A (x, y): 0x788086096da9e2f75ea0b93dca147c0dc4ade62831f77b38cad6d993a5759b47 0xb5834f296a12430b32f88b8c1d6d07e15a129316184c11a42998e9e2dd7f4ef4

Shared Secret B (x, y): 0x788086096da9e2f75ea0b93dca147c0dc4ade62831f77b38cad6d993a5759b47 0xb5834f296a12430b32f88b8c1d6d07e15a129316184c11a42998e9e2dd7f4ef4

Shared secret:  123465d27ebf3d3546a66ca0671dddf640f0cc5ccb6807b8f55e91f949c82986


### Encrypt with shared secret - Publisher

In [7]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
import os

# Generate a unique IV for each trace
def generate_iv():
    iv_length = 16  # IV length in bytes (recommended 16 bytes for AES)
    iv = os.urandom(iv_length)
    return iv

# Encrypt data using AES-CBC with a unique IV for each trace
def encrypt_data(key, iv, data):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()

    # Pad the data
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data) + padder.finalize()

    # Encrypt the padded data
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return ciphertext

# Example usage
key = bytes.fromhex('05bb88dde58728d36ef20a116decd35cdbda24a4c2798050074d76c23fc47a73')

# Original traces
traces = [
    b'"trace_id": "011", "timestamp": "2013-11-02T00:00:00Z", "C(0)/P(1)": "0", "value": 58.0',
    b'"trace_id": "012", "timestamp": "2013-11-02T00:15:00Z", "C(0)/P(1)": "0", "value": 75.0',
    b'"trace_id": "013", "timestamp": "2013-11-02T00:30:00Z", "C(0)/P(1)": "0", "value": 65.0',
    b'"trace_id": "014", "timestamp": "2013-11-02T00:45:00Z", "C(0)/P(1)": "0", "value": 0.08',
    b'"trace_id": "015", "timestamp": "2013-11-02T01:00:00Z", "C(0)/P(1)": "0", "value": 67.0',
    b'"trace_id": "016", "timestamp": "2013-11-02T01:15:00Z", "C(0)/P(1)": "0", "value": 69.0',
    b'"trace_id": "017", "timestamp": "2013-11-02T01:30:00Z", "C(0)/P(1)": "0", "value": 0.07',
    b'"trace_id": "018", "timestamp": "2013-11-02T01:45:00Z", "C(0)/P(1)": "0", "value": 73.0',
    b'"trace_id": "019", "timestamp": "2013-11-02T02:00:00Z", "C(0)/P(1)": "0", "value": 68.0',
    b'"trace_id": "0110", "timestamp": "2013-11-02T02:15:00Z", "C(0)/P(1)": "0", "value": 0.06'
]

# Encrypt and store each trace individually
encrypted_traces = []
for trace in traces:
    iv = generate_iv()
    ciphertext = encrypt_data(key, iv, trace)
    encrypted_trace = {
        'encrypted_data': ciphertext.decode('latin-1'),
        'iv': iv.decode('latin-1')
    }
    encrypted_traces.append(encrypted_trace)



In [8]:
encrypted_traces

[{'encrypted_data': '\x9dæ\x15@©¯µû\x9d¼Åë\x92\x00;C½ñ\x08ÁõÉ\x82NBJé\x08Ñ©\x8e\x1dyÝVn\x9b$s´ådfõ\x9eÄ~\x05¨\x9b\x19±p/ÞÚ×®\r°\x0cÛÜ !BmùµÄW`&½\x95Ö\x1eb-ÁþïOÖ±Ì\x9dÀS;ë\x89ÝX!Ñ',
  'iv': '5\xad<Òp\x8al1ÛBS;hòyç'},
 {'encrypted_data': 'qäv\tðj\x06ó¼ñ´\x963\x93éõ³}ØÈ´\x94Ì\x8a\x8a\x112½vÅ&VÌBªô½ßP]·§ë.6´\x01bF0gà\x13c\x7fÁ¥°\x08\x7frË¤ú\x1c0§\x16\x13¡$BÂz\xadcû5²3k*n;GHWsì!R\x95ÓYù\x00',
  'iv': "\x91ëºB\x82«¨¢Ê\x1c'V¹Õ^ò"},
 {'encrypted_data': '\x01Ù\x02Å\tò`\x83ñì½¢;Ú\x81\x93uh57{\x9c¼Dù8Él\x84\x13iâ\x00\x14ÂØa\x80\x9c-\x1dê±G(¹\xadO}\x8a\x8fê\x98æjxÈ\x02Z]ÎÙ¥ÛR\x14j)DauÌh\x8d°\x9dåÅ\x16\xa0\x82JßqRåÐ-}\x90Î\x93\x95\x87ó×',
  'iv': 'ºJv©Àdóåû!\x14\x93Æô~/'},
 {'encrypted_data': '¹X\x82e \x00\x97P\x92\x87n\x9aÎ\x85\x03\x17\x11Ä\x08V\x14Äfh`ªü©*Û&\x05ø\x81ü¯\x19ÁN{#\x7fò\xad\tð\x84$¬7\x17\x88\\É\x0f/5)þ\x14ßÍ\x05\x96Uç¡pô\x9fx#\x1c\x12¦¤Õe{ôì¨úQV}b\x0eø\x83î9+È\x9cL',
  'iv': '\x02ÿ\x03=T-e!\x1f\x99\x0eÉ\x95\x1eMb'},
 {'encrypted_data': 'ããÂ}ÿL_®=\x19\x0fÉ«pLñù=\x93Î\x85&Dèn½UÂ\\\x06¬g

### Decrypt - Subscriber

In [9]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

# Decrypt the encrypted data using AES-CBC
def decrypt_data(key, iv, ciphertext):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()

    # Decrypt the ciphertext
    decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()

    # Unpad the decrypted data
    unpadder = padding.PKCS7(128).unpadder()
    unpadded_data = unpadder.update(decrypted_data) + unpadder.finalize()
    return unpadded_data

In [10]:
key = bytes.fromhex('05bb88dde58728d36ef20a116decd35cdbda24a4c2798050074d76c23fc47a73') #llave compartida previamente por el canal seguro

decrypted_data_valid_traces = []
# Decrypt and print each trace
for encrypted_trace in encrypted_traces:
    ciphertext = encrypted_trace['encrypted_data'].encode('latin-1')
    iv = encrypted_trace['iv'].encode('latin-1')
    decrypted_data = decrypt_data(key, iv, ciphertext)
    decrypted_data_valid_traces.append(decrypted_data.decode('latin-1'))
    print("Decrypted Data:", decrypted_data.decode())

Decrypted Data: "trace_id": "011", "timestamp": "2013-11-02T00:00:00Z", "C(0)/P(1)": "0", "value": 58.0
Decrypted Data: "trace_id": "012", "timestamp": "2013-11-02T00:15:00Z", "C(0)/P(1)": "0", "value": 75.0
Decrypted Data: "trace_id": "013", "timestamp": "2013-11-02T00:30:00Z", "C(0)/P(1)": "0", "value": 65.0
Decrypted Data: "trace_id": "014", "timestamp": "2013-11-02T00:45:00Z", "C(0)/P(1)": "0", "value": 0.08
Decrypted Data: "trace_id": "015", "timestamp": "2013-11-02T01:00:00Z", "C(0)/P(1)": "0", "value": 67.0
Decrypted Data: "trace_id": "016", "timestamp": "2013-11-02T01:15:00Z", "C(0)/P(1)": "0", "value": 69.0
Decrypted Data: "trace_id": "017", "timestamp": "2013-11-02T01:30:00Z", "C(0)/P(1)": "0", "value": 0.07
Decrypted Data: "trace_id": "018", "timestamp": "2013-11-02T01:45:00Z", "C(0)/P(1)": "0", "value": 73.0
Decrypted Data: "trace_id": "019", "timestamp": "2013-11-02T02:00:00Z", "C(0)/P(1)": "0", "value": 68.0
Decrypted Data: "trace_id": "0110", "timestamp": "2013-11-02T02: